In [1]:
import krippendorff
import numpy as np
import pandas as pd
import re
import pickle
import random
import json
from collections import Counter

In [4]:
df_part_1 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_part_1.csv")

In [5]:
df_part_1

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,ChatGPT-4 5 classes aggregated,ChatGPT-3.5 5 classes,ChatGPT-3.5 3 classes,ChatGPT-3.5 5 classes aggregated,Llama3.3-70B-instruct 5 classes,Llama3.3-70B-instruct 3 classes,Llama3.3-70B-instruct 5 classes aggregated
0,Text 1: We will abolish long-term tuition fees...,Surface contradiction,100.0,"['Surface contradiction', 'Surface contradicti...",Inconsistent,Surface contradiction,Surface contradiction,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Surface contradiction,Inconsistent,Inconsistent
1,Text 1: The term of a prime minister should be...,Surface contradiction,40.0,"['Consistent', 'Surface contradiction', 'Indir...",Inconsistent,Consistent,Consistent,Consistent,Surface contradiction,Inconsistent,Inconsistent,Consistent,Consistent,Consistent
2,Text 1: Food that is free from genetic modific...,Consistent,40.0,"['Consistent', 'Consistent', 'Indirect inconsi...",Consistent,Consistent,Consistent,Consistent,Indirect inconsistency,Consistent,Inconsistent,Consistent,Consistent,Consistent
3,Text 1: The 5% hurdle prevents the representat...,Surface contradiction,60.0,"['Surface contradiction', 'Surface contradicti...",Inconsistent,Surface contradiction,Surface contradiction,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Surface contradiction,Inconsistent,Inconsistent
4,Text 1: Germany should consistently deport for...,Unrelated,100.0,"['Unrelated', 'Unrelated', 'Unrelated', 'Unrel...",Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,"ChatCompletionMessage(content=""Label: Unrelate...",Unrelated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.0,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Inconsistent,Inconsistent
344,Text 1: Students should have the free choice b...,Surface contradiction,60.0,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Consistent,Inconsistent
345,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.0,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Consistent,Consistent,Consistent
346,Text 1: The level of social basic security mus...,Factual inconsistency,40.0,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Inconsistent,Inconsistent


In [28]:
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_with_verification_05_10_24.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_with_training_inattentive_removed.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_cat_training_specialized_attentive_plus_trick.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_cat_training_specialized_top_3 - survey_5_cat_training_specialized.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_categories_training_explanations_best_nov_4.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_cat_training_explanations_best_6_nov_4.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_training_explanations_7_ppl_nov_5.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_cat_training _explanations_8_nov_7.csv")
# df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_5_categories_with_explanations_Nov_23.csv")
df = pd.read_csv("/Users/nursulusagimbayeva/Downloads/ContraDetect/survey_3_cat_26_11.csv")

In [29]:
df['Q26']

0            Your Prolific ID:
1    {"ImportId":"QID88_TEXT"}
2     5f59275d9a49e2140887ec41
3     6718b9492c331fe6c69f1666
4      671b5e696d6ac526141c00a
5     66da0e619af6e27e9e82bc75
6     6062d142077e1d0ec0c9361a
7     660c040de20b3bcf651ad3bb
8     63d13b6997850b99a55523e8
Name: Q26, dtype: object

In [283]:
# df = df[df['Q26'].isin(['5f59275d9a49e2140887ec41', '65a297db6bf9a9f5140b2b04', '6701715ac8d10cc870eb68aa', '67014cbab835dd54c54041e0'])]

In [134]:
# df = df[df['Q28.1'].notna()]
# df = df[df['Q54']=='Factual inconsistency']

In [30]:
df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,5,6,7,8,9,Q117,Q28,CorrectAnswers,Create New Field or Choose From Dropdown...,AttentionCheck
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Q5.\n\nText 1: We think that the nuclear power...,Q6.\n\nText 1: We believe that determining asy...,Q7.\n\nText 1: We believe that maintaining the...,Q8.\n\nText 1: We believe that the current gov...,Q9.\n\nText 1: We oppose expanding surveillanc...,Are there any categories that caused the most ...,Please give your feedback or comments on the t...,CorrectAnswers,Create New Field or Choose From Dropdown...,AttentionCheck
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID148""}","{""ImportId"":""QID152""}","{""ImportId"":""QID184""}","{""ImportId"":""QID186""}","{""ImportId"":""QID205""}","{""ImportId"":""QID206_TEXT""}","{""ImportId"":""QID49_TEXT""}","{""ImportId"":""CorrectAnswers""}","{""ImportId"":""Create New Field or Choose From D...","{""ImportId"":""AttentionCheck""}"
2,2024-11-25 9:50:41,2024-11-25 10:01:09,IP Address,86.152.200.240,100,627,TRUE,2024-11-25 10:01:10,R_2rGQrqPbVCMA4CZ,NaN,...,Indirect inconsistency,Consistent,Unrelated,Consistent,Unrelated,unrelated and consistent proved the trickiest ...,unable to copy and paste prolific number - had...,0,NaN,0
3,2024-11-25 11:39:16,2024-11-25 11:47:01,IP Address,86.152.171.43,100,465,TRUE,2024-11-25 11:47:01,R_8HZDfh2rE3QO9MM,NaN,...,Surface contradiction,Unrelated,Unrelated,Consistent,Unrelated,"No, I think everything is clear to me, now tha...",I always enjoy these tasks and find this study...,0,NaN,0
4,2024-11-25 12:01:08,2024-11-25 12:44:09,IP Address,80.41.77.205,100,2581,TRUE,2024-11-25 12:44:09,R_8MWAXGR9QjXd2AE,NaN,...,Indirect inconsistency,Consistent,Unrelated,Indirect inconsistency,Unrelated,no,NaN,0,NaN,0
5,2024-11-25 15:31:59,2024-11-25 15:45:10,IP Address,172.56.241.14,100,791,TRUE,2024-11-25 15:45:11,R_3mm8oS7n99dSKwo,NaN,...,Surface contradiction,Consistent,Unrelated,Consistent,Unrelated,No,There were one or two questions where I felt w...,0,NaN,0
6,2024-11-26 0:31:00,2024-11-26 0:37:48,IP Address,144.32.71.229,100,408,TRUE,2024-11-26 0:37:49,R_8KRdDPhAuZ56ci5,NaN,...,Surface contradiction,Consistent,Unrelated,Consistent,Unrelated,"In some cases, the two statements are unrelate...",NaN,0,NaN,0
7,2024-11-26 0:53:20,2024-11-26 0:58:36,IP Address,31.125.90.92,100,315,TRUE,2024-11-26 0:58:36,R_8dGeBPHUHs3Vb4x,NaN,...,Factual inconsistency,Consistent,Unrelated,Consistent,Unrelated,Not for the large part.,I don't have any recommendations for improveme...,0,NaN,0
8,2024-11-26 7:40:51,2024-11-26 7:49:06,IP Address,92.40.172.0,100,495,TRUE,2024-11-26 7:49:07,R_2HpCbdwFabpoOLy,NaN,...,Factual inconsistency,Consistent,Unrelated,Consistent,Unrelated,I was most undecided between indirect inconsis...,NaN,0,NaN,0


In [148]:
df_answers = df[[str(i) for i in range(1, 10)]]
# df_answers = df_answers[df_answers["1"].notna()]

In [149]:
df_answers = df_answers.reset_index(drop=True)

In [150]:
df_answers = df_answers.drop(index=[0,1])

In [151]:
df_answers = df_answers.reset_index(drop=True)

In [56]:
270/6

45.0

In [ ]:
20 unrelated

In [6]:
(270-20)/5

50.0

In [7]:
(250 - 30)/4

55.0

In [152]:
df_answers

,1,2,3,4,5,6,7,8,9
0,Unrelated,Surface contradiction,Consistent,Factual inconsistency,Indirect inconsistency,Consistent,Unrelated,Consistent,Unrelated
1,Unrelated,Factual inconsistency,Unrelated,Factual inconsistency,Surface contradiction,Unrelated,Unrelated,Consistent,Unrelated
2,Unrelated,Indirect inconsistency,Unrelated,Indirect inconsistency,Indirect inconsistency,Consistent,Unrelated,Indirect inconsistency,Unrelated
3,Unrelated,Factual inconsistency,Unrelated,Indirect inconsistency,Surface contradiction,Consistent,Unrelated,Consistent,Unrelated
4,Indirect inconsistency,Surface contradiction,Unrelated,Factual inconsistency,Surface contradiction,Consistent,Unrelated,Consistent,Unrelated
5,Unrelated,Factual inconsistency,Unrelated,Indirect inconsistency,Factual inconsistency,Consistent,Unrelated,Consistent,Unrelated
6,Unrelated,Indirect inconsistency,Consistent,Factual inconsistency,Factual inconsistency,Consistent,Unrelated,Consistent,Unrelated


In [226]:
# merged_df = pd.concat([df_answers, df2_answers], axis=0)

In [20]:
df_gold.columns

Index(['Input', 'Answer', 'Confidence', 'All answers', 'Answer_3_classes',
       'ChatGPT-4 5 classes', 'ChatGPT-4 3 classes',
       'Model_agreement_with_human', 'model_agreement_with_human_3_classes',
       'ChatGPT-3.5 5 classes', 'ChatGPT-3.5 3 classes', 'LLaMA 5 classes',
       'LLaMA 3 classes', 'LLaMA 70B 5 classes', 'LLaMA 70B 3 classes',
       'deberta-v3-large-nli_3_classes',
       'Model_agreement_with_human_gpt3.5_turbo',
       'model_agreement_with_human_3_classes_gpt3.5_turbo',
       'Model_agreement_with_human_LLaMA_8B',
       'Model_agreement_with_human_3_classes_LLaMA_8B',
       'Model_agreement_with_human_LLaMA_70B',
       'Model_agreement_with_human_3_classes_LLaMA_70B',
       'Model_agreement_with_human_3_classes_deberta_nli'],
      dtype='object')

In [54]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

In [60]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Expectation violation": 3, "Expectations violation": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

In [119]:
label_to_num = {'Level 1 (Surface contradiction)': 1, 'Level 2 (Meaning contradiction)': 2, 'Level 3 (Domain knowledge contradiction)': 3, 'Level 4 (Inconsistency in values/directions)': 4, 'Level 5 (Surprise/breaking stereotypes)': 5, 'Unrelated': 6, 'Consistent': 7}

In [166]:
matrix = []
for i in range(1, 10):
    res = []
    for answer in df_answers[str(i)]:
        res.append(label_to_num[answer])
    matrix.append(res)

In [167]:
# Example: 10 annotators, 8 questions, 7 ordinal labels (1-7)
# Rows = questions, Columns = annotators, Values = assigned labels
# Missing values can be represented by np.nan
matrix = np.array(matrix)

In [156]:
# Calculate Krippendorff's Alpha for ordinal data
alpha = krippendorff.alpha(reliability_data=matrix, level_of_measurement='ordinal')

print(f"Krippendorff's Alpha (Ordinal): {alpha}")

Krippendorff's Alpha (Ordinal): -0.08880241468212224


In [39]:
from statsmodels.stats.inter_rater import fleiss_kappa
from collections import Counter

In [168]:
matrix

array([[1, 1, 1, 1, 3, 1, 1],
       [3, 3, 3, 3, 3, 3, 3],
       [2, 1, 1, 1, 1, 1, 2],
       [3, 3, 3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3, 3, 3],
       [2, 1, 2, 2, 2, 2, 2],
       [1, 1, 1, 1, 1, 1, 1],
       [2, 2, 3, 2, 2, 2, 2],
       [1, 1, 1, 1, 1, 1, 1]])

In [100]:
# q [0, 1, 0, 2, 3] - question and what annotators said

In [159]:
matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6):
        res.append(counts[i])
    matrix_fleiss.append(res)

In [145]:
counts

Counter({np.int64(2): 6, np.int64(3): 1})

In [160]:
matrix_fleiss = np.array(matrix_fleiss)

In [161]:
matrix_fleiss

array([[6, 0, 1, 0, 0],
       [0, 0, 2, 3, 2],
       [5, 2, 0, 0, 0],
       [0, 0, 3, 4, 0],
       [0, 0, 2, 2, 3],
       [1, 6, 0, 0, 0],
       [7, 0, 0, 0, 0],
       [0, 6, 1, 0, 0],
       [7, 0, 0, 0, 0]])

In [162]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))

Krippendorff's alpha for nominal metric:  0.48865979381443303
Krippendorff's alpha for interval metric:  0.8241633579126489


In [43]:
a = 0.4**2 + 0.275**2 + 0.325**2

In [50]:
0.20875/0.65875

0.31688804554079697

In [163]:
# Example: Annotator counts for 7 labels (columns), for 8 questions (rows)
# Each row sums to the total number of annotators (e.g., 10)

# Calculate Fleiss' Kappa
fleiss_kappa_score = fleiss_kappa(matrix_fleiss, method='fleiss')
print(f"Fleiss' Kappa: {fleiss_kappa_score}")

Fleiss' Kappa: 0.48041237113402063


In [ ]:
Krippendorff's alpha for nominal metric:  0.0788912579957356
Krippendorff's alpha for interval metric:  0.24906600249066002
Fleiss' Kappa: 0.06427048431312825


can we improve perfromance by looking at individuals?
per row VS per column agreement 
right wing VS left wing 
check a

### Analyze Part I of the study

In [33]:
v1 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V1.csv")
v2 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V2.csv")
v3 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V3.csv")
v4 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V4.csv")
v5 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V5.csv")
v6 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V6.csv")
v7 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V7.csv")
v8 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V8.csv")
v9 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V9.csv")
v10 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_1\V10.csv")

In [34]:
answers = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10]

sample_to_responses = dict()
for i, df in enumerate(answers):
    print(i+1)
    for column in df.columns:
        string = df[column].loc[0]
        if re.match(r"^Q\d+", string):
            txt1, txt2 = df[column].loc[0].split("\n\n")[1], df[column].loc[0].split("\n\n")[2]
            label = df[column].iloc[-1]
            try:
                sample_to_responses[txt1 + "\n\n" + txt2].append(label)
            except:
                sample_to_responses[txt1 + "\n\n" + txt2] = [label]
          

1
2
3
4
5
6
7
8
9
10


In [35]:
len(sample_to_responses)

350

In [ ]:
12.21 

In [ ]:
['Factual inconsistency', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Indirect inconsistency', 'Indirect inconsistency', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction']


In [36]:
for el in sample_to_responses:
    if len(sample_to_responses[el]) != 5:
        print(el)
        print(sample_to_responses[el])

Text 1: We advocate for implementing stricter animal protection regulations to ensure livestock have permanent access to outdoor areas.

Text 2: We are working on increasing the production capacity of indoor-only livestock farms to meet the rising demand for meat.
['Factual inconsistency', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Indirect inconsistency', 'Indirect inconsistency', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction']
Text 1: The bargaining parties, unions, and employers should set wages, not the state by law.

Text 2: Companies with state contracts must be required to pay a higher minimum wage.
['Surface contradiction', 'Unrelated', 'Indirect inconsistency', 'Factual inconsistency']
Text 1: Companies with state contracts must be required to pay a higher minimum wage. 

Text 2: The bargaining parties, unions, and employers should set wages, not the state by law.
['Surface contradiction']


In [15]:
str_to_pop = '''Text 1: The bargaining parties, unions, and employers should set wages, not the state by law.

Text 2: Companies with state contracts must be required to pay a higher minimum wage.'''

In [16]:
sample_to_responses[str_to_pop].append("Surface contradiction")

In [ ]:
# with open("sample_to_responses_Part_1.pkl", 'wb') as f:
#     pickle.dump(sample_to_responses, f)

2 repeating statements

In [37]:
with open("sample_to_responses_Part_1.pkl", 'rb') as f:
    sample_to_responses = pickle.load(f)

In [46]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

In [30]:
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in sample_to_responses:
    res = []
    for answer in sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)

In [31]:
matrix = np.array(matrix)

In [32]:
matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 4):
        res.append(counts[i])
    matrix_fleiss.append(res)

In [33]:
matrix_fleiss = np.array(matrix_fleiss)

In [34]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))

Krippendorff's alpha for nominal metric:  0.5322713537483243
Krippendorff's alpha for interval metric:  0.5279549298276063


In [20]:
fleiss_kappa_score = fleiss_kappa(matrix_fleiss, method='fleiss')
print(f"Fleiss' Kappa: {fleiss_kappa_score}")

Fleiss' Kappa: 0.5320023896044189


### Results if we use 3 classes 

- Fleiss' Kappa: 0.5320023896044189
- Krippendorff's alpha for nominal metric:  0.5322713537483243
- Krippendorff's alpha for interval metric:  0.5279549298276065

348 samples (1 repeated, 1 over-annotated)

In [41]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

In [36]:
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in sample_to_responses:
    res = []
    for answer in sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)

In [37]:
matrix = np.array(matrix)

In [38]:
matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6):
        res.append(counts[i])
    matrix_fleiss.append(res)

In [39]:
matrix_fleiss = np.array(matrix_fleiss)

In [40]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))
fleiss_kappa_score = fleiss_kappa(matrix_fleiss, method='fleiss')
print(f"Fleiss' Kappa: {fleiss_kappa_score}")

Krippendorff's alpha for nominal metric:  0.3261605228488026
Krippendorff's alpha for interval metric:  0.5669791887209765
Fleiss' Kappa: 0.32577303608793373


### Results if we use 5 classes

- Krippendorff's alpha for nominal metric:  0.3261605228488026
- Krippendorff's alpha for interval metric:  0.5669791887209765
- Fleiss' Kappa: 0.32577303608793373

### Analyze part 2

In [2]:
v1 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V1.csv")
v2 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V2.csv")
v3 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V3.csv")
v4 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V4.csv")
v5 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V5.csv")
v6 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V6.csv")
v7 = pd.read_csv(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V7.csv')
v8 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V8.csv")
v9 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V9.csv")
v10 = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V10.csv")

Analysis assuming all answers are ready

In [20]:
answers = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10]
df_to_participant = {v1.to_string(): 'v1', v2.to_string(): 'v2', v3.to_string(): 'v3', v4.to_string(): 'v4', v5.to_string(): 'v5', v6.to_string(): 'v6', v7.to_string(): 'v7', v8.to_string(): 'v8', v9.to_string(): 'v9', v10.to_string(): 'v10'}

sample_to_participants = dict()
sample_to_responses = dict()
for i, df in enumerate(answers):
    print(i+1)
    for column in df.columns:
        string = df[column].loc[0]
        if re.match(r"^Q\d+", string):
            try:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n\n")[2]
            except:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
            label = df[column].iloc[-1]
            try:
                sample_to_responses[txt1 + "\n\n" + txt2].append(label)
                sample_to_participants[txt1 + "\n\n" + txt2].append(df_to_participant[df.to_string()])
            except:
                sample_to_responses[txt1 + "\n\n" + txt2] = [label]
                sample_to_participants[txt1 + "\n\n" + txt2] = [df_to_participant[df.to_string()]]

1
2
3
4
5
6
7
8
9
10


In [21]:
# for el in sample_to_responses:
#     if len(sample_to_responses[el]) == 3:
#         print(el)
#         print(sample_to_responses[el])

In [22]:
answers_str = ["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10"]
for answer in answers_str:
    print(answer)
    df = pd.read_csv(rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2_leftovers\{answer.capitalize()}.csv")
    for column in df.columns:
        string = df[column].loc[0]
        if re.match(r"^Q\d+", string):
            try:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n\n")[2]
                txt1, txt2 = txt1.replace("  ", " "), txt2.replace("  ", " ")
            except:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
                txt1, txt2 = txt1.replace("  ", " "), txt2.replace("  ", " ")
            label = df[column].iloc[-1]
            try:
                sample_to_responses[txt1 + "\n\n" + txt2].append(label)
                sample_to_participants[txt1 + "\n\n" + txt2].append(answer)
            except:
                sample_to_responses[txt1 + "\n\n" + txt2] = [label]
                sample_to_participants[txt1 + "\n\n" + txt2] = [answer]

v1
v2
v3
v4
v5
v6
v7
v8
v9
v10


In [23]:
df_last = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2_leftovers\2q_small.csv")

In [25]:
#add id particip.
answer = 'v1'
for column in df_last.columns:
    string = df_last[column].loc[0]
    if re.match(r"^Q\d+", string):
        try:
            txt1, txt2 = string.split("\n\n\n")[1], string.split("\n\n\n")[2]
            txt1, txt2 = txt1.replace("  ", " "), txt2.replace("  ", " ")
        except:
            txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
            txt1, txt2 = txt1.replace("  ", " "), txt2.replace("  ", " ")
        label = df_last[column].iloc[-1]
        try:
            sample_to_responses[txt1 + "\n\n" + txt2].append(label)
            sample_to_participants[txt1 + "\n\n" + txt2].append(answer)
        except:
            sample_to_responses[txt1 + "\n\n" + txt2] = [label]
            sample_to_participants[txt1 + "\n\n" + txt2] = [answer]

In [26]:
sample_to_responses.pop('a) We support social benefits for unemployed people.\n\nb) We are against financial aid for single mothers.')
sample_to_responses.pop('a) We will provide extensive social benefits. \nb) We will minimize all the taxes.\n\nb) We will minimize all the taxes.')
sample_to_responses.pop('a) We oppose sending weapons to war zones. \nb) We voted in favor of sending 50 tanks to a country that was attacked.\n\nb) We voted in favor of sending 50 tanks to a country that was attacked.')

sample_to_participants.pop('a) We support social benefits for unemployed people.\n\nb) We are against financial aid for single mothers.')
sample_to_participants.pop('a) We will provide extensive social benefits. \nb) We will minimize all the taxes.\n\nb) We will minimize all the taxes.')
sample_to_participants.pop('a) We oppose sending weapons to war zones. \nb) We voted in favor of sending 50 tanks to a country that was attacked.\n\nb) We voted in favor of sending 50 tanks to a country that was attacked.')

['v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v10',
 'v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v8',
 'v9',
 'v10']

In [27]:
final_sample_to_responses = dict()
for el in sample_to_responses:
    # look at each label and store only the first entry by each participant
    # TODO: check if we really add the first entry?
    labels = sample_to_responses[el]
    annotators = sample_to_participants[el]
    
        # Dictionary to store the first answer of each annotator
    first_answers = {}

    # Iterate over participants and labels
    for participant, label in zip(annotators, labels):
        if participant not in first_answers:  # Check if this is the first answer
            first_answers[participant] = label

    # Convert the results back into lists (if needed)
    unique_participants = list(first_answers.keys())
    if len(unique_participants) < 5: 
        print(unique_participants)
        print(el)
    unique_labels = list(first_answers.values())
    final_sample_to_responses[el] = unique_labels
    


Now no el-s with less than 5 annotations.

In [16]:
more_than_5 = 0
for el in final_sample_to_responses:
    if len(final_sample_to_responses[el]) > 5:
        more_than_5 += 1
        print(el)
        print(final_sample_to_responses[el])
        final_sample_to_responses[el].remove(random.choice(final_sample_to_responses[el]))
        
    elif len(final_sample_to_responses[el]) < 5:
        final_sample_to_responses[el].append(random.choice(list(label_to_num.keys())))

In [28]:
# with open("sample_to_responses_Part_2_some_6.pkl", 'wb') as f:
#     pickle.dump(final_sample_to_responses, f)

In [29]:
with open("sample_to_responses_Part_1.pkl", 'rb') as f:
    sample_to_responses_pt1 = pickle.load(f)

In [30]:
final_sample_to_responses.update(sample_to_responses_pt1)

In [40]:
# with open("all_sample_to_responses_some_6.pkl", 'wb') as f:
#     pickle.dump(final_sample_to_responses, f)

In [111]:
len(final_sample_to_responses)

698

In [61]:
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)
    else:
        print(sample)
        print(res)
        matrix.append(random.sample(res, 5))


Text 1: We fully support the conversion of additional car streets to bicycle streets.

Text 2: We propose reducing the taxes on the purchase of new cars to stimulate the automotive industry
[1, 3, 3, 3, 3, 3]
Text 1: The UN should have more power to enforce resolutions.

Text 2: National sovereignty should not be undermined by international bodies.
[5, 5, 5, 5, 4, 5]
Text 1: The extraction of gas from rock layers (fracking) is to be completely banned.

Text 2: Fracking poses a potential threat to groundwater.
[2, 2, 2, 2, 2, 2]
Text 1: There should be a general vaccination requirement against Covid-19.

Text 2: European agricultural subsidies must be significantly reduced.
[1, 1, 1, 1, 1, 1]
Text 1: To combat terrorism, the armed forces should also be deployable domestically.

Text 2: The state should financially support the installation of solar systems on rooftops.
[1, 1, 1, 1, 1, 1]
Text 1: The surveillance state is undemocratic and is meant to intimidate people.

Text 2: Video surv

In [35]:
len(matrix)

698

In [62]:
matrix = np.array(matrix)

In [63]:
matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6): # 6 if all labels, 4 if merged
        res.append(counts[i])
    matrix_fleiss.append(res)

In [64]:
matrix_fleiss = np.array(matrix_fleiss)

In [65]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))

Krippendorff's alpha for nominal metric:  0.31292666775787403
Krippendorff's alpha for interval metric:  0.5294496706504368


In [53]:
fleiss_kappa_score = fleiss_kappa(matrix_fleiss, method='fleiss')
print(f"Fleiss' Kappa: {fleiss_kappa_score}")

Fleiss' Kappa: 0.3131921084413967


Filling in missing vals

In [6]:
answers = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10]
df_to_participant = {v1.to_string(): 'v1', v2.to_string(): 'v2', v3.to_string(): 'v3', v4.to_string(): 'v4', v5.to_string(): 'v5', v6.to_string(): 'v6', v7.to_string(): 'v7', v8.to_string(): 'v8', v9.to_string(): 'v9', v10.to_string(): 'v10'}

sample_to_participants = dict()
sample_to_responses = dict()
for i, df in enumerate(answers):
    print(i+1)
    for column in df.columns:
        string = df[column].loc[0]
        if re.match(r"^Q\d+", string):
            try:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n\n")[2]
            except:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
            label = df[column].iloc[-1]
            try:
                sample_to_responses[txt1 + "\n\n" + txt2].append(label)
                sample_to_participants[txt1 + "\n\n" + txt2].append(df_to_participant[df.to_string()])
            except:
                sample_to_responses[txt1 + "\n\n" + txt2] = [label]
                sample_to_participants[txt1 + "\n\n" + txt2] = [df_to_participant[df.to_string()]]
                
            
          

1
2
3
4
5
6
7
8
9
10


Pop practice questions 

In [7]:
sample_to_responses.pop('a) We support social benefits for unemployed people.\n\nb) We are against financial aid for single mothers.')
sample_to_responses.pop('a) We will provide extensive social benefits. \nb) We will minimize all the taxes.\n\nb) We will minimize all the taxes.')
sample_to_responses.pop('a) We oppose sending weapons to war zones. \nb) We voted in favor of sending 50 tanks to a country that was attacked.\n\nb) We voted in favor of sending 50 tanks to a country that was attacked.')

sample_to_participants.pop('a) We support social benefits for unemployed people.\n\nb) We are against financial aid for single mothers.')
sample_to_participants.pop('a) We will provide extensive social benefits. \nb) We will minimize all the taxes.\n\nb) We will minimize all the taxes.')
sample_to_participants.pop('a) We oppose sending weapons to war zones. \nb) We voted in favor of sending 50 tanks to a country that was attacked.\n\nb) We voted in favor of sending 50 tanks to a country that was attacked.')

['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10']

In [20]:
participants = set(['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10'])

In [25]:
question_to_not_answered_participant = dict()
question_repeated_trial = dict()
counter = 0
for el in sample_to_responses:
    answered = set(sample_to_participants[el])
    if len(answered) < 5:
        counter += 1
        print(el)
        values = sample_to_responses[el]
        print(values)
        annotators = sample_to_participants[el]
        print(annotators)
        # Initialize an empty dictionary
        annot_to_value = {}
        # Iterate through both lists using zip
        for annotator, value in zip(annotators, values):
            # Add the value to the corresponding participant key
            annot_to_value.setdefault(annotator, []).append(value)
        filtered_annot_to_value = {key: value for key, value in annot_to_value.items() if len(value) > 1}
        if len(filtered_annot_to_value) > 0:
            question_repeated_trial[el] = filtered_annot_to_value
        not_answered = participants - answered
        still_needed = 5 - len(answered)
        question_to_not_answered_participant[el] = (list(not_answered), still_needed)

Text 1: Motivated and confident workforce is a crucial success factor for the economy.

Text 2: We support limiting the scope of employee influence in decision-making within companies to promote competitiveness.
['Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']
['v1', 'v1', 'v1', 'v2', 'v2', 'v2', 'v10']
Text 1: We want broad support for agriculture with no limitations.

Text 2: We support the exclusive promotion of organic agriculture.
['Consistent', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Factual inconsistency']
['v1', 'v1', 'v2', 'v2', 'v4', 'v4', 'v7']
Text 1: There should be a €29 ticket for everyone to use local public transport.

Text 2: Passengers need to pass a background-check and have stable income to qualify for the discounted ticket subscription.
['Indirect 

In [31]:
# with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\question_repeated_trial.pkl', 'wb') as f:
#     pickle.dump(question_repeated_trial, f)

In [2]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\question_repeated_trial.pkl', 'rb') as f:
    question_repeated_trial = pickle.load(f)

In [2]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [ ]:
# we want participant -> class -> number of repeated times

In [23]:
participant_to_repeated_sample_by_class = dict() 

In [24]:
for i in range(1, 11):
    participant_to_repeated_sample_by_class[f'v{i}'] = dict()

In [25]:
for i in range(10):
    for annotator in participant_to_repeated_sample_by_class:
        participant_to_repeated_sample_by_class[annotator]['total_repeated'] = 0  
        for class_name in classes_5:
            participant_to_repeated_sample_by_class[annotator][class_name] = dict()
            participant_to_repeated_sample_by_class[annotator][class_name]['2_switches'] = []
            participant_to_repeated_sample_by_class[annotator][class_name]['3_switches'] = []
            participant_to_repeated_sample_by_class[annotator][class_name]['4_switches'] = []

In [26]:
# look at each sample
for sample in question_repeated_trial:
    # look at each annotator
    for annotator in question_repeated_trial[sample]:
        # look how often they were offered the same question, count together
        total_switch = len(question_repeated_trial[sample][annotator])
        participant_to_repeated_sample_by_class[annotator]['total_repeated'] += total_switch
        # check the class they chose initially
        orig_class = question_repeated_trial[sample][annotator][0]
        # check all the times they switched in this sample
        for j in range(1, total_switch, 1):
            participant_to_repeated_sample_by_class[annotator][orig_class][f'{j+1}_switches'].append(question_repeated_trial[sample][annotator][j])

In [33]:
len(df_gold[df_gold['Explanations'] != str(0)])

237

In [27]:
participant_to_repeated_sample_by_class

{'v1': {'total_repeated': 40,
  'Unrelated': {'2_switches': ['Unrelated', 'Consistent', 'Unrelated'],
   '3_switches': ['Unrelated'],
   '4_switches': ['Unrelated']},
  'Consistent': {'2_switches': ['Surface contradiction',
    'Consistent',
    'Consistent',
    'Consistent'],
   '3_switches': [],
   '4_switches': []},
  'Indirect inconsistency': {'2_switches': ['Indirect inconsistency',
    'Factual inconsistency',
    'Surface contradiction',
    'Indirect inconsistency',
    'Indirect inconsistency',
    'Indirect inconsistency'],
   '3_switches': ['Indirect inconsistency'],
   '4_switches': []},
  'Factual inconsistency': {'2_switches': ['Factual inconsistency'],
   '3_switches': [],
   '4_switches': []},
  'Surface contradiction': {'2_switches': ['Surface contradiction',
    'Indirect inconsistency',
    'Surface contradiction',
    'Indirect inconsistency'],
   '3_switches': ['Surface contradiction'],
   '4_switches': []}},
 'v2': {'total_repeated': 40,
  'Unrelated': {'2_switch

In [29]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\participant_to_repeated_sample_by_class.json', 'w') as f:
    json.dump(participant_to_repeated_sample_by_class, f)

In [26]:
len(question_repeated_trial)

190

In [40]:
change_per_class = dict()
# divide into groups. answered 2 times, 3 times, 4 times etc.
change_per_class['2_times'] = dict()
change_per_class['3_times'] = dict()
change_per_class['4_times'] = dict()

for num_times in change_per_class:
    for class_name in ['Unrelated', 'Consistent', 'Indirect inconsistency', 'Factual inconsistency', 'Surface contradiction']:
        change_per_class[num_times][class_name] = {}
        change_per_class[num_times][class_name]['answers'] = []
        change_per_class[num_times][class_name]['annotators'] = []
        

In [41]:
counter_2_times_change = 0
counter_3_times_change = 0
counter_4_times_change = 0
for el in question_repeated_trial:
    for annotator in question_repeated_trial[el]:
        answers = question_repeated_trial[el][annotator]
        if len(answers) == 2:
            counter_2_times_change += 1
            change_per_class['2_times'][answers[0]]['answers'].append(answers[1])
            change_per_class['2_times'][answers[0]]['annotators'].append(annotator)
        elif len(answers) == 3:
            counter_3_times_change += 1
            change_per_class['3_times'][answers[0]]['answers'].append((answers[1], answers[2]))
            change_per_class['3_times'][answers[0]]['annotators'].append(annotator)
        elif len(answers) == 4:
            counter_4_times_change += 1
            change_per_class['4_times'][answers[0]]['answers'].append((answers[1], answers[2], answers[3]))
            change_per_class['4_times'][answers[0]]['annotators'].append(annotator)
        else:
            print(len(answers))

In [42]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\change_per_class.pkl', 'wb') as f:
    pickle.dump(change_per_class, f)

Let's analyze Unrelated Class

In [ ]:
change_per_class

- Unrelated

2 times

27 total 

Counter({'Unrelated': 22, 'Consistent': 3, 'Factual inconsistency': 2})

Change rate 18.51851851851852% 

- Consistent

2 times

33 total

Counter({'Consistent': 30, 'Surface contradiction': 1, 'Unrelated': 1, 'Indirect inconsistency': 1})

Change rate 9.090909090909092%


- Indirect inconsistency

61 total

Counter({'Indirect inconsistency': 47, 'Factual inconsistency': 9, 'Surface contradiction': 3, 'Consistent': 2})

Change rate 22.950819672131146%


- Factual

28 total

Counter({'Factual inconsistency': 13, 'Indirect inconsistency': 10, 'Surface contradiction': 4, 'Consistent': 1})

Change rate 53.57142857142857%


- Surface 

61 total

Change rate 19.672131147540984%


Annotators who switched

Unrelated
{'v10': 9, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
Consistent
{'v10': 8, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
Indirect inconsistency
{'v10': 5, 'v5': 4, 'v3': 2, 'v6': 2, 'v4': 2, 'v1': 1, 'v7': 1, 'v8': 1}
Factual inconsistency
{'v10': 6, 'v3': 4, 'v7': 2, 'v4': 1, 'v6': 1, 'v8': 1}
Surface contradiction
{'v10': 8, 'v5': 4, 'v6': 3, 'v7': 3, 'v3': 2, 'v8': 2, 'v1': 1, 'v4': 1}

In [59]:
12/61*100

19.672131147540984

In [58]:
print(len(change_per_class['2_times']['Surface contradiction']['answers']))
print(Counter(change_per_class['2_times']['Surface contradiction']['answers']))


61
Counter({'Surface contradiction': 49, 'Factual inconsistency': 6, 'Indirect inconsistency': 4, 'Unrelated': 2})


In [63]:
for class_name in change_per_class['2_times']:
    print(class_name)
    answers = change_per_class['2_times']['Factual inconsistency']['answers']
    annotators = change_per_class['2_times']['Factual inconsistency']['annotators']

    # Step 1: Initialize a dictionary to track switches
    switch_count = {}

    # Step 2: Iterate over the data and count non-"Factual inconsistency" answers
    for annotator, answer in zip(annotators, answers):
        if answer != class_name:  # Check if it's a "switch"
            switch_count[annotator] = switch_count.get(annotator, 0) + 1

    # Step 3: Display the results
    sorted_switch_count = dict(sorted(switch_count.items(), key=lambda item: item[1], reverse=True))
    print(sorted_switch_count)

Unrelated
{'v10': 9, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
Consistent
{'v10': 8, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
Indirect inconsistency
{'v10': 5, 'v5': 4, 'v3': 2, 'v6': 2, 'v4': 2, 'v1': 1, 'v7': 1, 'v8': 1}
Factual inconsistency
{'v10': 6, 'v3': 4, 'v7': 2, 'v4': 1, 'v6': 1, 'v8': 1}
Surface contradiction
{'v10': 8, 'v5': 4, 'v6': 3, 'v7': 3, 'v3': 2, 'v8': 2, 'v1': 1, 'v4': 1}


In [65]:
for class_name in change_per_class['3_times']:
    print(class_name)
    answers = change_per_class['3_times']['Factual inconsistency']['answers']
    annotators = change_per_class['3_times']['Factual inconsistency']['annotators']

    # Step 1: Initialize a dictionary to track switches
    switch_count = {}

    # Step 2: Iterate over the data and count non-"Factual inconsistency" answers
    for annotator, answer in zip(annotators, answers):
        if answer[0] != class_name:  # Check if it's a "switch"
            switch_count[annotator] = switch_count.get(annotator, 0) + 1

    # Step 3: Display the results
    sorted_switch_count = dict(sorted(switch_count.items(), key=lambda item: item[1], reverse=True))
    print(sorted_switch_count)

Unrelated
{'v10': 2, 'v4': 1, 'v5': 1, 'v6': 1, 'v9': 1}
Consistent
{'v10': 2, 'v4': 1, 'v5': 1, 'v6': 1}
Indirect inconsistency
{'v10': 2, 'v4': 1, 'v5': 1, 'v6': 1, 'v9': 1}
Factual inconsistency
{'v10': 1, 'v9': 1}
Surface contradiction
{'v4': 1, 'v5': 1, 'v6': 1, 'v9': 1, 'v10': 1}


In [27]:
question_repeated_trial

{'Text 1: Motivated and confident workforce is a crucial success factor for the economy.\n\nText 2: We support limiting the scope of employee influence in decision-making within companies to promote competitiveness.': {'v1': ['Indirect inconsistency',
   'Indirect inconsistency',
   'Indirect inconsistency'],
  'v2': ['Indirect inconsistency',
   'Indirect inconsistency',
   'Indirect inconsistency']},
 'Text 1: We want broad support for agriculture with no limitations.\n\nText 2: We support the exclusive promotion of organic agriculture.': {'v1': ['Consistent',
   'Surface contradiction'],
  'v2': ['Surface contradiction', 'Surface contradiction'],
  'v4': ['Surface contradiction', 'Surface contradiction']},
 'Text 1: There should be a €29 ticket for everyone to use local public transport.\n\nText 2: Passengers need to pass a background-check and have stable income to qualify for the discounted ticket subscription.': {'v1': ['Indirect inconsistency',
   'Factual inconsistency'],
  'v2

In [7]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_7.txt", 'r', encoding='utf-8') as f:
    v7_text = f.read()

In [ ]:
# reduce the questions that will be anyway answered in V7
for el in question_to_not_answered_participant: 
    text1 = el.split("\n\n")[0].split(": ")[1]
    text2 = el.split("\n\n")[1].split(": ")[1]
    if text1 in v7_text and text2 in v7_text:
        # print("Same question used!")
        answer = question_to_not_answered_participant[el]
        question_to_not_answered_participant[el] = (answer[0], answer[1]-1)

In [9]:
# How many questions left to annotate?
total = 0
for el in question_to_not_answered_participant:
    total += question_to_not_answered_participant[el][1]
    
# everyone gets 366//10 = 36 questions, 6 people get 37

In [10]:
total

366

In [54]:
all_participants = list(map(lambda x: True if x >= 41 else False, items))

In [58]:
if set(all_participants) == set([False]):
    print("yes")

yes


In [10]:
def meets_conditions(selected_participants, new_questions_per_participants):
    # Condition: the number of q-s per participant shouldn't be more than 37
    all_participants = [len(new_questions_per_participants[participant]) for participant in selected_participants]
    all_participants = list(map(lambda x: True if x >= 41 else False, all_participants))
    if set(all_participants) == set([True]):
        # print("all are more than 41")
        return True
    for participant in selected_participants:
        if len(new_questions_per_participants[participant]) >= 41:
            return False
    return True

In [15]:
def all_are_40_or_41(new_questions_per_participants):
    result = [len(new_questions_per_participants[el]) for el in new_questions_per_participants]
    map_result = list(map(lambda x: True if 45 >= x >= 36 else False, result))
    if set(map_result) == set([True]):
        return True
    return False

In [17]:
new_questions_per_participants = dict()
for annotator in participants:
    new_questions_per_participants[annotator] = []
    
counter = 0
# select randomly whom to assign to annotate. keep the same number of q-s for everyone.
for el in question_to_not_answered_participant:
    # print(counter)
    to_select = question_to_not_answered_participant[el][1]
    lst_participants = question_to_not_answered_participant[el][0]
    # print(to_select)
    # print(lst_participants)

    if to_select != 0:
        while True:   
            selected_participants = random.sample(lst_participants, to_select)
            if meets_conditions(selected_participants, new_questions_per_participants):
                break
        for annotator in selected_participants:
            new_questions_per_participants[annotator].append(el)
    counter += 1


In [22]:
for el in new_questions_per_participants:
    text = new_questions_per_participants[el]
    with open(rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\part_2_leftovers\{el}.json", 'w', encoding='utf-8') as f:
        json.dump(text, f)

In [ ]:
question_to_not_answered_participant

In [30]:
new_questions_per_participants['v10'][16]

'Text 1: We need guest worker programs to fill labor shortages.\n\nText 2: Mass immigration dilutes a nation’s cultural identity.'

In [ ]:
from v2
5, 10, 18 to v10

from v6
15, 37 to v10
23 to v9

from v8 
11, 17 to v10





In [53]:
for el in new_questions_per_participants:
    with open(rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\part_2_leftovers\{el}.json", 'rb') as f:
        res = json.load(f)
        
    print(el, ":", len(res))

v6 : 41
v1 : 41
v2 : 41
v3 : 40
v8 : 41
v4 : 41
v10 : 41
v5 : 40
v9 : 40


In [ ]:
to v10:
take 7 from  
v2 - 3 -  
v6 - 3 - done
v8 - 2

to v9:
take 1


In [18]:
def meets_conditions_new_participant(new_participant, lst_participants, new_questions_per_participants):
    # Condition: the number of q-s per participant shouldn't be more than 37
    all_participants = [len(new_questions_per_participants[participant]) for participant in lst_participants]
    all_participants = list(map(lambda x: True if x >= 41 else False, all_participants))
    if set(all_participants) == set([True]):
        print("all are more than 41")
        return True
    if len(new_questions_per_participants[new_participant]) >= 41:
        return False
    return True

In [23]:
for index, text in enumerate(new_questions_per_participants['v4']):
    if question_to_not_answered_participant[text][1] > 1:
        lst_participants = question_to_not_answered_participant[text][0]
        print(lst_participants)
        while True:   
            new_participant = random.choice(lst_participants)
            print("new_participant", new_participant)
            if meets_conditions_new_participant(new_participant, lst_participants, new_questions_per_participants):
                break
        new_questions_per_participants[new_participant].append(new_questions_per_participants['v4'].pop(index))
        
        

['v6', 'v8', 'v1', 'v5', 'v2']
new_participant v5
new_participant v1
new_participant v6
['v4', 'v2', 'v8', 'v5', 'v1', 'v9', 'v10']
new_participant v5
new_participant v2
new_participant v4
['v6', 'v4', 'v8', 'v1', 'v5', 'v2', 'v10']
new_participant v6
['v6', 'v4', 'v2', 'v5', 'v1', 'v9']
new_participant v2
new_participant v1
new_participant v2
new_participant v2
new_participant v2
new_participant v4
['v6', 'v4', 'v8', 'v1', 'v2', 'v10']
new_participant v4
['v3', 'v4', 'v9', 'v1', 'v2', 'v10']
new_participant v4
['v3', 'v6', 'v4', 'v2', 'v1', 'v9', 'v10']
new_participant v6
['v3', 'v6', 'v4', 'v2', 'v1', 'v9', 'v10']
new_participant v2
new_participant v9
['v3', 'v4', 'v2', 'v8', 'v1', 'v9', 'v10']
new_participant v3
new_participant v8
['v3', 'v4', 'v2', 'v5', 'v1', 'v9', 'v10']
new_participant v3
new_participant v1
new_participant v9
['v3', 'v6', 'v4', 'v9', 'v5', 'v1', 'v2', 'v10']
new_participant v1
new_participant v2
new_participant v2
new_participant v4
['v3', 'v6', 'v4', 'v2', 'v5'

In [6]:
counter = 0
for el in sample_to_responses:
    if len(sample_to_responses[el]) > 5:
        counter += 1
        print(el)
        print(sample_to_responses[el])
        print(sample_to_participants[el])


Text 1: Motivated and confident workforce is a crucial success factor for the economy.

Text 2: We support limiting the scope of employee influence in decision-making within companies to promote competitiveness.
['Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']
['v1', 'v1', 'v1', 'v2', 'v2', 'v2', 'v10']
Text 1: We want broad support for agriculture with no limitations.

Text 2: We support the exclusive promotion of organic agriculture.
['Consistent', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction']
['v1', 'v1', 'v2', 'v2', 'v4', 'v4']
Text 1: In the unresolved issue of waste storage, it is irresponsible to further increase the waste.

Text 2: We support the extension of nuclear power plant operating times.
['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency

In [7]:
len(sample_to_responses)

349

In [8]:
counter

73

73 questions have more than 5 answers. but they are not unique (same people). 

- 65 questions have exactly 5 annotators.

- 14 questions have more than 5 annotators

- 270 have less than 5 annotators

- 168 have less than 4 annotators

- 50 less than 3

- 7 less than 2

### Compute gold labels for Part I.

In [66]:
for el in final_sample_to_responses:
    if len(final_sample_to_responses[el]) != 5:
        print(el)
        print(final_sample_to_responses[el])

Text 1: We fully support the conversion of additional car streets to bicycle streets.

Text 2: We propose reducing the taxes on the purchase of new cars to stimulate the automotive industry
['Unrelated', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']
Text 1: The UN should have more power to enforce resolutions.

Text 2: National sovereignty should not be undermined by international bodies.
['Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Factual inconsistency', 'Surface contradiction']
Text 1: The extraction of gas from rock layers (fracking) is to be completely banned.

Text 2: Fracking poses a potential threat to groundwater.
['Consistent', 'Consistent', 'Consistent', 'Consistent', 'Consistent', 'Consistent']
Text 1: There should be a general vaccination requirement against Covid-19.

Text 2: European agricultural subsidies must be significantly redu

One sample with many annotations, including repeated by the same person.

In [45]:
to_pop = '''Text 1: We advocate for implementing stricter animal protection regulations to ensure livestock have permanent access to outdoor areas.

Text 2: We are working on increasing the production capacity of indoor-only livestock farms to meet the rising demand for meat.'''

sample_to_responses.pop(to_pop)

['Factual inconsistency',
 'Surface contradiction',
 'Surface contradiction',
 'Surface contradiction',
 'Surface contradiction',
 'Indirect inconsistency',
 'Indirect inconsistency',
 'Surface contradiction',
 'Surface contradiction',
 'Surface contradiction']

In [67]:
input = list(final_sample_to_responses.keys())
answer = [] 
confidence = []
all_answers = []

for el in final_sample_to_responses:
    counter = Counter(final_sample_to_responses[el])
    #  TODO: check how it chooses most common
    most_common_element, count = counter.most_common(1)[0]
    answer.append(most_common_element)
    confidence_score = round(count/(len(final_sample_to_responses[el]))*100, 3)
    all_answers.append(final_sample_to_responses[el])
    confidence.append(confidence_score)

what if some class is always competing with another, and then counter.most_common chooses alphabetically, so Factual Inconsistency will be overrepresented in comp. with Indirect inconsistency 

-> choose randomly if 2 or more with the same count

most common chooses whatever appeared earlier so it is already quite random.

E.g. counter = Counter(['b', 'c', 'c','a', 'a'])
will choose ('c', 2) and not 'a'

In [68]:
df_gold = pd.DataFrame({'Input': input, 'Answer': answer, 'Confidence': confidence, 'All answers': all_answers})

In [69]:
df_gold['Answer_3_classes'] = 0

df_gold['Answer_3_classes'] = df_gold['Answer'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

In [70]:
Counter(df_gold['Confidence'])

Counter({60.0: 257,
         40.0: 168,
         80.0: 142,
         100.0: 109,
         50.0: 10,
         83.333: 6,
         66.667: 2,
         33.333: 2,
         20.0: 1,
         70.0: 1})

In [71]:
# df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

In [2]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

### Eval ChatGPT-4, 5 types

In [2]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_5_types.pkl', 'rb') as f:
    chatgpt_1 = pickle.load(f)
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_5_types_350_700.pkl', 'rb') as f:
    chatgpt_2 = pickle.load(f)

In [9]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in chatgpt_2.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in chatgpt_2:
        chatgpt_2[new_key] = chatgpt_2.pop(old_key)

In [12]:
chatgpt_all = dict()
chatgpt_all.update(chatgpt_1)
chatgpt_all.update(chatgpt_2)

In [16]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent
...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent


In [17]:
df_gold['ChatGPT-4 5 classes'] = 0

In [18]:
predictions = []
true_labels = []
for el in chatgpt_all:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = chatgpt_all[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 5 classes'] = pred
    except:
        print(el)
    
# for el in chatgpt_2:
#     pred = chatgpt_2[el]['label']
#     predictions.append(pred)
#     true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
#     true_labels.append(true_label)
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_119332\1598067405.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Surface contradiction' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 5 classes'] = pred


In [19]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types.pkl', 'rb') as f:
    answers_chatgpt_3_types = pickle.load(f)

In [22]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types_350_700.pkl', 'rb') as f:
    answers_chatgpt_3_types_pt2 = pickle.load(f)

In [83]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt4_3_types_leftovers.pkl', 'rb') as f:
    answers_chatgpt4_3_types_leftovers = pickle.load(f)

In [25]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_chatgpt_3_types_pt2.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_chatgpt_3_types_pt2:
        answers_chatgpt_3_types_pt2[new_key] = answers_chatgpt_3_types_pt2.pop(old_key)

In [27]:
answers_chatgpt_3_types_all = dict()
answers_chatgpt_3_types_all.update(answers_chatgpt_3_types)
answers_chatgpt_3_types_all.update(answers_chatgpt_3_types_pt2)

In [85]:
answers_chatgpt_3_types_all.update(answers_chatgpt4_3_types_leftovers)

In [87]:
df_gold['ChatGPT-4 3 classes'] = 0
predictions = []
true_labels = []
for el in answers_chatgpt_3_types_all:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer_3_classes'].iloc[0]
        true_labels.append(true_label)
        pred = answers_chatgpt_3_types_all[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 3 classes'] = pred
    except:
        print(el)
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_119332\658612249.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Surface contradiction' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 3 classes'] = pred


In [91]:
wrong_to_correct_label_5_types = {"Factually inconsistent": "Factual inconsistency", 
                                  'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                  'Surface contradiction': 'Surface contradiction',
                                  'Factual inconsistency': 'Factual inconsistency',
                                  'Indirect inconsistency': 'Indirect inconsistency'
                                  }
 
wrong_to_correct_label_3_types = {"Indirect inconsistency": "Inconsistent", "Factual inconsistency": "Inconsistent",
                                  "Surface contradiction": "Inconsistent", 'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                   'Inconsistent':  'Inconsistent',
                                   'Indirect Inconsistency': 'Inconsistent'}
 

In [89]:
Counter(df_gold['ChatGPT-4 3 classes'])

Counter({'Inconsistent': 189,
         'Indirect inconsistency': 185,
         'Unrelated': 138,
         'Consistent': 130,
         'Surface contradiction': 32,
         'Factual inconsistency': 21,
         'Indirect Inconsistency': 3})

In [92]:
df_gold['ChatGPT-4 3 classes'] = list(map(lambda x: wrong_to_correct_label_3_types[x], df_gold['ChatGPT-4 3 classes']))

In [61]:
df_gold['ChatGPT-4 5 classes'] = list(map(lambda x: wrong_to_correct_label_5_types[x], df_gold['ChatGPT-4 5 classes']))

In [93]:
Counter(df_gold['ChatGPT-4 3 classes'])

Counter({'Inconsistent': 430, 'Unrelated': 138, 'Consistent': 130})

In [164]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

Format answers so that they are not misread

### Accuracy Part 1 ChatGPT-4 

5 classes is 0.5374

3 classes with different prompt Accuracy is 0.2471. It probably misunderstood the assignment, and even tho it was asked to output 3 classes, it did 5.

If we aggregate over classes: 
Accuracy: 0.8563

### Accuracy Part 1 ChatGPT-3.5 

5 classes is 0.3724 - 341 samples (TODO: fix parsing)

If we aggregate over classes: 
Accuracy: 0.7040

3 classes prompt:
Accuracy: 0.1026


### Accuracy Part 1 Llama3.3 70B Instruct 

5 classes is 0.4751 - 341 samples (TODO: fix parsing)

If aggregared: 
Accuracy: 0.7989

3 classes prompt: Accuracy: 0.1554


### Accuracy Part 1 Llama3 8B Instruct

5 classes: 0.3255 - 341 samples (TODO: fix parsing)

If aggregated: Accuracy: 0.5747

### Accuracy Part 1 Mixtral_8_7B_Instruct_v0_1t

5 classes: 0.0176

If aggregated: 0.0172




### Measure accuracy

In [98]:
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['ChatGPT-4 3 classes'])

In [99]:
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.8009


##### Per class accuracy

In [8]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']

In [104]:
model_acc_per_class = dict()

In [105]:
model_acc_per_class['chatgpt4'] = {}

In [106]:
model_acc_per_class['chatgpt4']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['ChatGPT-4 5 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt4'][class_name] = accuracy
    

Unrelated
Accuracy: 0.6695
Consistent
Accuracy: 0.8854
Indirect inconsistency
Accuracy: 0.7151
Factual inconsistency
Accuracy: 0.1721
Surface contradiction
Accuracy: 0.3279


In [108]:
model_acc_per_class['chatgpt4']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['ChatGPT-4 3 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt4'][class_name] = accuracy

Unrelated
Accuracy: 0.7119
Consistent
Accuracy: 0.8438
Inconsistent
Accuracy: 0.8140


#### Pretend model is 6th annotator

In [4]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

In [135]:
label_to_num = {'Unrelated': 1, "Consistent": 2,  "Inconsistent": 3, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

In [15]:
model_as_6th_annotator = dict()

In [27]:
model_name = "LLaMA 70B"
model_as_6th_annotator[model_name] = dict()
model_as_6th_annotator[model_name]['5_classes'] = dict()

#### 5 classes first ####
print("5 classes", model_name)
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)
    
for el in final_sample_to_responses:
    model_label = df_gold[df_gold['Input'] == el]['LLaMA 70B 5 classes'].iloc[0]
    final_sample_to_responses[el].append(model_label)
    
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 6:
        matrix.append(res)
    else:
        # print(sample)
        # print(res)
        matrix.append(random.sample(res, 6))


matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6): # 6 if all labels, 4 if merged
        res.append(counts[i])
    matrix_fleiss.append(res)
    
matrix_fleiss = np.array(matrix_fleiss)

print(f"Krippendorff's alpha for nominal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print(f"Krippendorff's alpha for interval metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss))
print(f"Krippendorff's alpha for ordinal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal'))

model_as_6th_annotator[model_name]['5_classes']['krippendorff_nominal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='nominal')
model_as_6th_annotator[model_name]['5_classes']['krippendorff_interval'] = krippendorff.alpha(value_counts=matrix_fleiss)
model_as_6th_annotator[model_name]['5_classes']['krippendorff_ordinal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal')

5 classes LLaMA 70B
Krippendorff's alpha for nominal metric, LLaMA 70B:  0.30712717700787917
Krippendorff's alpha for interval metric, LLaMA 70B:  0.5178868699550001
Krippendorff's alpha for ordinal metric, LLaMA 70B:  0.5211202232875967


In [32]:
model_name = 'Deberta'
model_as_6th_annotator[model_name] = dict()

In [33]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Inconsistent": 3, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

model_as_6th_annotator[model_name]['3_classes'] = dict()

#### 3 classes ####
print("3 classes", model_name)

with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)
    
for el in final_sample_to_responses:
    model_label = df_gold[df_gold['Input'] == el]['deberta-v3-large-nli_3_classes'].iloc[0]
    final_sample_to_responses[el].append(model_label)
    
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 6:
        matrix.append(res)
    else:
        # print(sample)
        # print(res)
        matrix.append(random.sample(res, 6))


matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 4): 
        res.append(counts[i])
    matrix_fleiss.append(res)
    
matrix_fleiss = np.array(matrix_fleiss)

print(f"Krippendorff's alpha for nominal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print(f"Krippendorff's alpha for interval metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss))
print(f"Krippendorff's alpha for ordinal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal'))

model_as_6th_annotator[model_name]['3_classes']['krippendorff_nominal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='nominal')
model_as_6th_annotator[model_name]['3_classes']['krippendorff_interval'] = krippendorff.alpha(value_counts=matrix_fleiss)
model_as_6th_annotator[model_name]['3_classes']['krippendorff_ordinal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal')

3 classes Deberta
Krippendorff's alpha for nominal metric, Deberta:  0.4407783097442195
Krippendorff's alpha for interval metric, Deberta:  0.38196297921900035
Krippendorff's alpha for ordinal metric, Deberta:  0.42413601948129365


In [38]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_as_6th_annotator.pkl", 'wb') as f:
#     pickle.dump(model_as_6th_annotator, f)

Calculate for humans

In [35]:
model_name = "Humans"
model_as_6th_annotator[model_name] = dict()
model_as_6th_annotator[model_name]['5_classes'] = dict()

#### 5 classes first ####
print("5 classes", model_name)
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)
    
matrix = []
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)
    else:
        # print(sample)
        # print(res)
        matrix.append(random.sample(res, 5))


matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6): # 6 if all labels, 4 if merged
        res.append(counts[i])
    matrix_fleiss.append(res)
    
matrix_fleiss = np.array(matrix_fleiss)

print(f"Krippendorff's alpha for nominal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print(f"Krippendorff's alpha for interval metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss))
print(f"Krippendorff's alpha for ordinal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal'))

model_as_6th_annotator[model_name]['5_classes']['krippendorff_nominal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='nominal')
model_as_6th_annotator[model_name]['5_classes']['krippendorff_interval'] = krippendorff.alpha(value_counts=matrix_fleiss)
model_as_6th_annotator[model_name]['5_classes']['krippendorff_ordinal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal')

5 classes Humans
Krippendorff's alpha for nominal metric, Humans:  0.31279874939868946
Krippendorff's alpha for interval metric, Humans:  0.5297441220108765
Krippendorff's alpha for ordinal metric, Humans:  0.5279433730298616


In [36]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Inconsistent": 3, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

model_as_6th_annotator[model_name]['3_classes'] = dict()

#### 3 classes ####
print("3 classes", model_name)

with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)
    
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)
    else:
        # print(sample)
        # print(res)
        matrix.append(random.sample(res, 5))


matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 4): 
        res.append(counts[i])
    matrix_fleiss.append(res)
    
matrix_fleiss = np.array(matrix_fleiss)

print(f"Krippendorff's alpha for nominal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print(f"Krippendorff's alpha for interval metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss))
print(f"Krippendorff's alpha for ordinal metric, {model_name}: ", krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal'))

model_as_6th_annotator[model_name]['3_classes']['krippendorff_nominal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='nominal')
model_as_6th_annotator[model_name]['3_classes']['krippendorff_interval'] = krippendorff.alpha(value_counts=matrix_fleiss)
model_as_6th_annotator[model_name]['3_classes']['krippendorff_ordinal'] = krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal')

3 classes Humans
Krippendorff's alpha for nominal metric, Humans:  0.5073399681871671
Krippendorff's alpha for interval metric, Humans:  0.4739017596113717
Krippendorff's alpha for ordinal metric, Humans:  0.5012670885089561


### Agreement per class for humans

In [4]:
#### 5 classes first ####

print("5 classes")
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)

5 classes


In [14]:
samples_unrelated = list(df_gold[df_gold['Answer']=='Factual inconsistency']['Input'])

matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in samples_unrelated:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 5:
        matrix.append(res)
    else:
        matrix.append(random.sample(res, 5))
        
matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 6): 
        res.append(counts[i])
    matrix_fleiss.append(res)
    
matrix_fleiss = np.array(matrix_fleiss)

print(f"Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print(f"Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))
print(f"Krippendorff's alpha for ordinal metric: ", krippendorff.alpha(value_counts=matrix_fleiss, level_of_measurement='ordinal'))


Krippendorff's alpha for nominal metric:  -0.09014437096788663
Krippendorff's alpha for interval metric:  0.008731250554717462
Krippendorff's alpha for ordinal metric:  0.005087288857259731


#### What is a probability that model agrees with indiv. (model - random human for each question)

for this question it agrees with 3/6 annotators -> avg agreement

per question, per class, overall


In [150]:
model_agreement_with_human = []
for i in range(len(df_gold)):
    human_answers = eval(df_gold.iloc[i]['All answers'])
    model_answer = df_gold.iloc[i]['ChatGPT-4 5 classes']
    model_agreement = Counter(human_answers)[model_answer]/len(human_answers)
    model_agreement_with_human.append(model_agreement)

In [152]:
Counter(model_agreement_with_human)

Counter({0.2: 150,
         0.4: 123,
         0.6: 119,
         0.0: 100,
         1.0: 97,
         0.8: 90,
         0.8333333333333334: 5,
         0.16666666666666666: 5,
         0.5: 5,
         0.3333333333333333: 3,
         0.6666666666666666: 1})

In [158]:
df_gold['Model_agreement_with_human'] = model_agreement_with_human

In [159]:
model_agreement_with_human_3_classes = []
for i in range(len(df_gold)):
    human_answers = eval(df_gold.iloc[i]['All answers'])
    human_answers = [wrong_to_correct_label_3_types[el] for el in human_answers]
    model_answer = df_gold.iloc[i]['ChatGPT-4 3 classes']
    model_agreement = Counter(human_answers)[model_answer]/len(human_answers)
    model_agreement_with_human_3_classes.append(model_agreement)

In [162]:
Counter(model_agreement_with_human_3_classes)

Counter({1.0: 344,
         0.8: 130,
         0.6: 73,
         0.4: 49,
         0.2: 45,
         0.0: 44,
         0.8333333333333334: 6,
         0.16666666666666666: 5,
         0.5: 2})

In [163]:
df_gold['model_agreement_with_human_3_classes'] = model_agreement_with_human_3_classes

#### Per class agreement with humans

In [168]:
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human']))/len(df_class)
    print(avg_agreement)

Unrelated
0.5932203389830506
Consistent
0.7413194444444442
Indirect inconsistency
0.49273743016759736
Factual inconsistency
0.26530054644808726
Surface contradiction
0.36630236794171245


In [169]:
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    avg_agreement = sum(list(df_class['model_agreement_with_human_3_classes']))/len(df_class)
    print(avg_agreement)

Unrelated
0.6703389830508473
Consistent
0.740972222222222
Inconsistent
0.7789256198347121


#### Performance in comparison to humans

In [106]:
df_gold_part_1['ChatGPT-4 5 classes aggregated'] = 0

df_gold_part_1['ChatGPT-4 5 classes aggregated'] = df_gold_part_1['ChatGPT-4 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

In [116]:
predictions = list(df_gold_part_1["ChatGPT-4 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])

In [117]:
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8563


### Eval ChatGPT-3.5, 5 types

In [119]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_5_types.pkl', 'rb') as f:
    answers_chatgpt35_turbo_5_types = pickle.load(f)
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_3_types.pkl', 'rb') as f:
    answers_chatgpt35_turbo_3_types = pickle.load(f)

In [120]:
df_gold_part_1['ChatGPT-3.5 5 classes'] = 0
df_gold_part_1['ChatGPT-3.5 3 classes'] = 0
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = 0

In [134]:
el.split('\n')[0]

'Text 1: We support granting exceptions for clubs in the city center from nighttime noise restrictions.'

In [148]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_chatgpt35_turbo_3_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = answers_chatgpt35_turbo_3_types[el]['label']
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'ChatGPT-3.5 3 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\3383750435.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'ChatGPT-3.5 3 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\3383750435.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.1026


In [146]:
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = 0
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = df_gold_part_1['ChatGPT-3.5 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["ChatGPT-3.5 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7040


### Evaluate LLaMa

In [192]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_8B_instruct_5_types.pkl", 'rb') as f:
    answers_llama3_8B_instruct_5_types = pickle.load(f)

In [178]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_3_70B_instruct_3_types.pkl", 'rb') as f:
    answers_llama3_3_70B_instruct_3_types = pickle.load(f)

In [154]:
def extract_label_and_explanation(text):
    label_match = re.search(r"Label:(.*?)Explanation:", text, re.DOTALL)
    label_text = label_match.group(1).strip() if label_match else text

    # Extract text after "Explanation"
    explanation_match = re.search(r"Explanation:(.*)", text, re.DOTALL)
    explanation_text = explanation_match.group(1).strip() if explanation_match else text
    
    return label_text, explanation_text

In [193]:
df_gold_part_1['Llama3-8B-instruct 5 classes'] = 0
df_gold_part_1['Llama3-8B-instruct 3 classes'] = 0
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = 0

In [194]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_llama3_8B_instruct_5_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred, explanation = extract_label_and_explanation(str(answers_llama3_8B_instruct_5_types[el]))
        pred = pred.replace('\\n', '')
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Llama3-8B-instruct 5 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\2112758861.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Llama3-8B-instruct 5 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\2112758861.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.3255


In [195]:
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = 0
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = df_gold_part_1['Llama3-8B-instruct 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["Llama3-8B-instruct 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5747


### Evaluate Mistral

In [196]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_mixtral_8_7B_Instruct_v0_1t_5_types.pkl", 'rb') as f:
    answers_mixtral_8_7B_Instruct_v0_1t_5_types = pickle.load(f)

In [197]:
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 3 classes'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = 0

In [200]:
answers_mixtral_8_7B_Instruct_v0_1t_5_types

{"Text 1: The first degree should remain free of charge for everyone.\nText 2: Scholarship should be paid regardless of the parents' income.": ChatCompletionMessage(content=" Label: Consistent\n\nExplanation: Both statements support the idea that education should be accessible and affordable for everyone, regardless of their financial situation. The first statement suggests that education should be free of charge for everyone, while the second statement suggests that scholarships should be provided regardless of the parents' income. These statements are not contradictory, as they both advocate for making education more accessible and affordable.", role='assistant', function_call=None, tool_calls=None),
 'Text 1: Only states that consistently respect human rights should receive development aid.\nText 2: Trade relations with states that violate human rights should be stopped.': ChatCompletionMessage(content=' Label: Indirect inconsistency\nExplanation: The first statement implies that de

In [201]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_mixtral_8_7B_Instruct_v0_1t_5_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred, explanation = extract_label_and_explanation(str(answers_mixtral_8_7B_Instruct_v0_1t_5_types[el]))
        pred = pred.replace('\\n', '')
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Mixtral_8_7B_Instruct_v0_1t 5 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\614373465.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Mixtral_8_7B_Instruct_v0_1t 5 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\614373465.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.0176


In [205]:
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.0172


In [202]:
698 - count_fail

341

In [2]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")